# 📊 Notebook 1: Exploratory Data Analysis (EDA) and Data Understanding

**Author:** Amey Talkatkar  
**Email:** ameytalkatkar169@gmail.com  
**GitHub:** https://github.com/ameytrainer  
**Course:** MLOps with Agentic AI - Advanced Certification  

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Load and understand the sales dataset structure
- ✅ Perform comprehensive exploratory data analysis
- ✅ Identify data quality issues
- ✅ Discover patterns and trends in sales data
- ✅ Understand feature relationships and correlations
- ✅ Make data-driven decisions for feature engineering

---

## 🔥 The Problem (Why EDA Matters)

**Real-World Disaster Story:**

A data scientist was given a sales dataset and immediately jumped into training models:
- Trained XGBoost without looking at the data
- Got 95% accuracy! Celebrated! 🎉
- Deployed to production

**In production:**
- Model predicted negative sales
- Model predicted $10 million for a $50 item
- Model failed on weekends (no weekend data in training!)

**What went wrong?**
- Never checked data distributions
- Never found outliers
- Never understood seasonal patterns
- Never validated assumptions

**This is why EDA is CRITICAL before any modeling.**

---

## 📦 Setup and Imports

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

## 📂 Step 1: Load Data

First, let's generate and load our sales data.

In [ ]:
# Generate data if not exists
import subprocess
import os

data_path = '../data/raw/sales_data.csv'

if not os.path.exists(data_path):
    print("📊 Generating sales data...")
    result = subprocess.run(
        ['python', '../data/generate_data.py', '--rows', '10000', '--output', data_path],
        capture_output=True,
        text=True
    )
    print(result.stdout)
else:
    print(f"✅ Data already exists at: {data_path}")

# Load data
df = pd.read_csv(data_path, parse_dates=['date'])

print(f"\n✅ Data loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 🔍 Step 2: Initial Data Inspection

Let's understand the structure and content of our data.

In [ ]:
# First look at the data
print("📋 First 5 rows:")
display(df.head())

print("\n📋 Last 5 rows:")
display(df.tail())

In [ ]:
# Data types and memory usage
print("📊 Data Types and Memory Usage:")
print(df.info())

print(f"\n💾 Total Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Check for missing values
print("🔍 Missing Values:")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})

print(missing_df[missing_df['Missing Count'] > 0])

if missing.sum() == 0:
    print("✅ No missing values found!")
else:
    print(f"⚠️ Total missing values: {missing.sum()}")

## 📈 Step 3: Statistical Summary

Understanding the distribution of numerical features.

In [ ]:
# Numerical statistics
print("📊 Numerical Features Summary:")
display(df.describe())

# Additional percentiles
print("\n📊 Extended Percentiles:")
display(df[['price', 'quantity', 'sales']].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))

In [ ]:
# Categorical features
print("📊 Categorical Features:")
categorical_cols = ['region', 'product', 'category', 'season']

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    value_counts = df[col].value_counts()
    for value, count in value_counts.items():
        print(f"  {value:15s}: {count:6,} ({count/len(df)*100:5.2f}%)")

## 📊 Step 4: Distribution Analysis

Visualizing the distributions of key features.

In [ ]:
# Distribution of numerical features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Distribution of Numerical Features', fontsize=16, fontweight='bold')

# Price distribution
axes[0, 0].hist(df['price'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Price Distribution')
axes[0, 0].set_xlabel('Price ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['price'].mean(), color='red', linestyle='--', label=f'Mean: ${df["price"].mean():.2f}')
axes[0, 0].legend()

# Quantity distribution
axes[0, 1].hist(df['quantity'], bins=50, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Quantity Distribution')
axes[0, 1].set_xlabel('Quantity')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(df['quantity'].mean(), color='red', linestyle='--', label=f'Mean: {df["quantity"].mean():.1f}')
axes[0, 1].legend()

# Sales distribution
axes[0, 2].hist(df['sales'], bins=50, color='lightcoral', edgecolor='black')
axes[0, 2].set_title('Sales Distribution')
axes[0, 2].set_xlabel('Sales ($)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].axvline(df['sales'].mean(), color='red', linestyle='--', label=f'Mean: ${df["sales"].mean():,.2f}')
axes[0, 2].legend()

# Box plots for outlier detection
df.boxplot(column='price', ax=axes[1, 0])
axes[1, 0].set_title('Price Box Plot (Outlier Detection)')
axes[1, 0].set_ylabel('Price ($)')

df.boxplot(column='quantity', ax=axes[1, 1])
axes[1, 1].set_title('Quantity Box Plot')
axes[1, 1].set_ylabel('Quantity')

df.boxplot(column='sales', ax=axes[1, 2])
axes[1, 2].set_title('Sales Box Plot')
axes[1, 2].set_ylabel('Sales ($)')

plt.tight_layout()
plt.show()

print("💡 Insights:")
print(f"  - Price ranges from ${df['price'].min():.2f} to ${df['price'].max():.2f}")
print(f"  - Quantity ranges from {df['quantity'].min()} to {df['quantity'].max()}")
print(f"  - Sales range from ${df['sales'].min():.2f} to ${df['sales'].max():.2f}")
print(f"  - Check for outliers in box plots above")

## 📅 Step 5: Time Series Analysis

Understanding temporal patterns is crucial for sales forecasting.

In [ ]:
# Daily sales trend
daily_sales = df.groupby('date')['sales'].sum().reset_index()

plt.figure(figsize=(15, 6))
plt.plot(daily_sales['date'], daily_sales['sales'], linewidth=1, alpha=0.7)
plt.title('Daily Sales Trend Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.grid(True, alpha=0.3)

# Add rolling average
daily_sales['rolling_7d'] = daily_sales['sales'].rolling(window=7).mean()
daily_sales['rolling_30d'] = daily_sales['sales'].rolling(window=30).mean()
plt.plot(daily_sales['date'], daily_sales['rolling_7d'], linewidth=2, label='7-day MA', color='red')
plt.plot(daily_sales['date'], daily_sales['rolling_30d'], linewidth=2, label='30-day MA', color='green')
plt.legend()
plt.show()

print("💡 Insights:")
print("  - Look for trends (upward/downward)")
print("  - Identify seasonality patterns")
print("  - Check for anomalies or spikes")

In [ ]:
# Monthly sales analysis
monthly_sales = df.groupby('month')['sales'].agg(['sum', 'mean', 'count']).reset_index()
monthly_sales.columns = ['Month', 'Total Sales', 'Avg Sales', 'Num Transactions']

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Monthly total sales
axes[0].bar(monthly_sales['Month'], monthly_sales['Total Sales'], color='steelblue')
axes[0].set_title('Total Sales by Month', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Total Sales ($)')
axes[0].set_xticks(range(1, 13))
axes[0].grid(axis='y', alpha=0.3)

# Monthly average sales
axes[1].bar(monthly_sales['Month'], monthly_sales['Avg Sales'], color='coral')
axes[1].set_title('Average Sales by Month', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Average Sales ($)')
axes[1].set_xticks(range(1, 13))
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Monthly Statistics:")
display(monthly_sales)

# Find peak months
peak_month = monthly_sales.loc[monthly_sales['Total Sales'].idxmax(), 'Month']
print(f"\n💡 Peak sales month: {peak_month} (November/December = holiday season!)")

In [ ]:
# Day of week analysis
dow_sales = df.groupby('day_of_week')['sales'].agg(['sum', 'mean', 'count']).reset_index()
dow_sales['day_name'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Sales by day of week
axes[0].bar(dow_sales['day_name'], dow_sales['sum'], color='teal')
axes[0].set_title('Total Sales by Day of Week', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Day of Week')
axes[0].set_ylabel('Total Sales ($)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Weekend vs Weekday
weekend_sales = df[df['is_weekend'] == True]['sales'].sum()
weekday_sales = df[df['is_weekend'] == False]['sales'].sum()
axes[1].bar(['Weekday', 'Weekend'], [weekday_sales, weekend_sales], color=['lightblue', 'orange'])
axes[1].set_title('Weekday vs Weekend Sales', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Total Sales ($)')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Insights:")
print(f"  - Weekend sales: ${weekend_sales:,.2f}")
print(f"  - Weekday sales: ${weekday_sales:,.2f}")
print(f"  - Weekend boost: {(weekend_sales/weekday_sales - 1)*100:.1f}%")

## 🗺️ Step 6: Regional and Product Analysis

Understanding regional and product-level patterns.

In [ ]:
# Sales by region
region_sales = df.groupby('region')['sales'].agg(['sum', 'mean', 'count']).reset_index()
region_sales = region_sales.sort_values('sum', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Total sales by region
axes[0].barh(region_sales['region'], region_sales['sum'], color='skyblue')
axes[0].set_title('Total Sales by Region', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Total Sales ($)')
axes[0].grid(axis='x', alpha=0.3)

# Pie chart
axes[1].pie(region_sales['sum'], labels=region_sales['region'], autopct='%1.1f%%', startangle=90)
axes[1].set_title('Sales Distribution by Region', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Regional Statistics:")
display(region_sales)

In [ ]:
# Sales by product
product_sales = df.groupby('product')['sales'].agg(['sum', 'mean', 'count']).reset_index()
product_sales = product_sales.sort_values('sum', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Total sales by product
axes[0].barh(product_sales['product'], product_sales['sum'], color='lightgreen')
axes[0].set_title('Total Sales by Product', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Total Sales ($)')
axes[0].grid(axis='x', alpha=0.3)

# Average transaction size
axes[1].barh(product_sales['product'], product_sales['mean'], color='coral')
axes[1].set_title('Average Transaction Size by Product', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Avg Sales per Transaction ($)')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Product Statistics:")
display(product_sales)

## 🔗 Step 7: Correlation Analysis

Understanding relationships between features.

In [ ]:
# Correlation matrix
numeric_cols = ['price', 'quantity', 'sales', 'month', 'day_of_week']
correlation = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Key Correlations:")
print(f"  - Sales vs Price: {correlation.loc['sales', 'price']:.3f}")
print(f"  - Sales vs Quantity: {correlation.loc['sales', 'quantity']:.3f}")
print("\n📝 Note: Sales = Price × Quantity (perfect correlation expected!)")

In [ ]:
# Scatter plots for relationships
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Price vs Sales
axes[0].scatter(df['price'], df['sales'], alpha=0.3, s=10)
axes[0].set_title('Price vs Sales', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Price ($)')
axes[0].set_ylabel('Sales ($)')
axes[0].grid(True, alpha=0.3)

# Quantity vs Sales
axes[1].scatter(df['quantity'], df['sales'], alpha=0.3, s=10, color='green')
axes[1].set_title('Quantity vs Sales', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Quantity')
axes[1].set_ylabel('Sales ($)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📊 Step 8: Advanced Analysis - Seasonality Decomposition

In [ ]:
# Create time series for decomposition
daily_sales_ts = df.groupby('date')['sales'].sum().reset_index()
daily_sales_ts.set_index('date', inplace=True)

# Simple moving averages to identify trend and seasonality
daily_sales_ts['MA_7'] = daily_sales_ts['sales'].rolling(window=7).mean()
daily_sales_ts['MA_30'] = daily_sales_ts['sales'].rolling(window=30).mean()
daily_sales_ts['MA_90'] = daily_sales_ts['sales'].rolling(window=90).mean()

plt.figure(figsize=(15, 6))
plt.plot(daily_sales_ts.index, daily_sales_ts['sales'], alpha=0.3, label='Daily Sales')
plt.plot(daily_sales_ts.index, daily_sales_ts['MA_7'], linewidth=2, label='7-day MA')
plt.plot(daily_sales_ts.index, daily_sales_ts['MA_30'], linewidth=2, label='30-day MA')
plt.plot(daily_sales_ts.index, daily_sales_ts['MA_90'], linewidth=2, label='90-day MA (Trend)')
plt.title('Sales Trend with Moving Averages', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Sales ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Insights:")
print("  - 7-day MA: Captures weekly patterns")
print("  - 30-day MA: Smooths out weekly noise")
print("  - 90-day MA: Shows overall trend")

## ✅ Step 9: Data Quality Report

Summary of data quality checks.

In [ ]:
print("="*60)
print("DATA QUALITY REPORT")
print("="*60)

print(f"\n✅ Dataset Size: {len(df):,} rows × {len(df.columns)} columns")

print(f"\n✅ Missing Values: {df.isnull().sum().sum()} (0.00%)")

print(f"\n✅ Duplicate Rows: {df.duplicated().sum()} (0.00%)")

print(f"\n✅ Date Range:")
print(f"   Start: {df['date'].min().date()}")
print(f"   End:   {df['date'].max().date()}")
print(f"   Days:  {(df['date'].max() - df['date'].min()).days}")

print(f"\n✅ Numerical Ranges:")
print(f"   Price:    ${df['price'].min():.2f} - ${df['price'].max():.2f}")
print(f"   Quantity: {df['quantity'].min()} - {df['quantity'].max()}")
print(f"   Sales:    ${df['sales'].min():.2f} - ${df['sales'].max():.2f}")

print(f"\n✅ Categorical Values:")
print(f"   Regions:  {df['region'].nunique()} unique ({', '.join(df['region'].unique())})")
print(f"   Products: {df['product'].nunique()} unique ({', '.join(df['product'].unique())})")
print(f"   Seasons:  {df['season'].nunique()} unique ({', '.join(df['season'].unique())})")

print(f"\n✅ Business Logic Checks:")
calculated_sales = df['price'] * df['quantity']
sales_match = (calculated_sales - df['sales']).abs().max() < 0.01
print(f"   Sales = Price × Quantity: {'PASS ✅' if sales_match else 'FAIL ❌'}")

print("\n" + "="*60)
print("READY FOR FEATURE ENGINEERING!")
print("="*60)

## 🎓 Key Takeaways

### What We Learned:
1. ✅ **Data Structure**: 10,000+ rows with date, region, product, price, quantity, sales
2. ✅ **No Quality Issues**: No missing values, no duplicates, data ranges are reasonable
3. ✅ **Seasonality**: Strong Q4 boost (November-December holiday season)
4. ✅ **Day-of-Week Effect**: Weekend sales are higher than weekdays
5. ✅ **Regional Patterns**: Different regions have different product preferences
6. ✅ **Growth Trend**: Overall upward trend in sales over time

### Why This Matters for MLOps:
- 📊 **Feature Engineering**: We now know which patterns to capture (seasonality, day-of-week)
- 🔄 **Data Validation**: We'll create checks based on these ranges (price $10-$1000, quantity 1-100)
- 📈 **Model Selection**: Time series patterns suggest we need features for trends and seasonality
- 🎯 **Monitoring**: We know what "normal" looks like (for drift detection later)

---

## 🚀 Next Steps

**Next Notebook:** `02_Feature_Engineering.ipynb`

We'll use these insights to:
- Create lag features (yesterday's sales, last week's sales)
- Encode categorical variables (region, product)
- Create seasonal indicators
- Scale numerical features
- Split into train/test sets

---

**Author:** Amey Talkatkar | **Course:** MLOps with Agentic AI

**© 2024 Amey Talkatkar** | Educational Use License